## 引用相关套件

In [6]:
import argparse
import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
from skimage.viewer import ImageViewer
import sys

sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque

import json
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf

## 设定训练相关参数

In [7]:
GAME    = 'bird'             # 游戏Log名称
ACTIONS = 2                  # 动作种类 (0: Not Flap, 1: Flap)
GAMMA   = 0.99               # 动作衰退 decay
OBSERVATION       = 3200.    # 训练前观察的步骤
EXPLORE          = 3000000.  # 逐渐降低episilon的总步数
FINAL_EPSILON    = 0.0001    # epsilon最后的值
INITIAL_EPSILON  = 0.1       # epsilon的初始值
REPLAY_MEMORY    = 50000     # Replay 记忆最大长度
BATCH            = 32        # mini-batch的大小
FRAME_PER_ACTION = 1         # 每个动作对应到的帧 (Frame)
LEARNING_RATE    = 1e-4      # 学习率

img_rows , img_cols = 80, 80 # 输入图片大小

img_channels = 4             # 帧数量

## 训练网路

In [8]:
def trainNetwork(model,args):
    # 起始游戏
    game_state = game.GameState()

    # 将过去的观察记忆在经验回放纪录（Experience Replay）D中
    D = deque()

    #  什么都不做取得第一个状态并开始预处理80X80X4大小的图
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal = game_state.frame_step(do_nothing)

    """
    r_0 代表 reward
    r_0 分为三种：
    0.1 : 代表bird活着
    +1  : 代表bird通过了水管
    -1  : 代表bird死亡
    
    terminal是个布尔标志用来判断游戏结束与否
    
    """
    
    # 图像预处理
    """
    将彩色图像转换为灰阶图
    调整图像的大小为80 X 80像素
    提供4张图像给神经网络进行训练
    """
    x_t = skimage.color.rgb2gray(x_t)
    x_t = skimage.transform.resize(x_t,(80,80))
    x_t = skimage.exposure.rescale_intensity(x_t,out_range=(0,255))

    
    
    x_t = x_t / 255.0

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

    #调整成Keras能吃的数据格式
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4

    

    if args['mode'] == 'Run':
        OBSERVE = 999999999    #只观测，不训练
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #进入训练模式
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON

    t = 0
    while (True):
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([ACTIONS])
        # 采用epsilon greedy当成动作
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                # 随机动作
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t)       #输入四张图以取得预测结果
                # 采取最大Q值动作
                max_Q = np.argmax(q)
                action_index = max_Q
                a_t[max_Q] = 1

        # 渐渐降低epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # 执行动作并且观测出状态与奖励
        x_t1_colored, r_t, terminal = game_state.frame_step(a_t)
        """
        r_t 代表 reward
        r_t 分为三种：
        0.1 : 代表bird活著
        +1  : 代表bird通过了水管
        -1  : 代表bird死亡
        """

        # 图像预处理
        """
        将彩色图像转换为灰阶图
        调整图像的大小为80 X 80像素
        提供4张图像给神经网络进行训练
        """
        x_t1 = skimage.color.rgb2gray(x_t1_colored)
        x_t1 = skimage.transform.resize(x_t1,(80,80))
        x_t1 = skimage.exposure.rescale_intensity(x_t1, out_range=(0, 255))


        x_t1 = x_t1 / 255.0


        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # 所有的episode(s,a,r,s′)都会存储在重放表格d中
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #结束观测后只单纯训练

        if t > OBSERVE:
            # 采样mini-batch做资料训练
            minibatch = random.sample(D, BATCH)

            # 经验回放 (Exprience Replay) 
            state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch) # (s,a,r,s′)
            state_t = np.concatenate(state_t)
            state_t1 = np.concatenate(state_t1)
            targets = model.predict(state_t)
            Q_sa = model.predict(state_t1)
            targets[range(BATCH), action_t] = reward_t + GAMMA*np.max(Q_sa, axis=1)*np.invert(terminal)

            loss += model.train_on_batch(state_t, targets)

        s_t = s_t1
        t = t + 1

        #每一千次存储模型
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # 印出相关资讯
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

## 建立 Convolution Neural Network

In [14]:
def buildmodel():
    model = Sequential()
    #80*80*4 => 20*20*32 
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(img_rows,img_cols,img_channels)))  
    model.add(Activation('relu'))
    #20*20*32 => 10*10*64
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    #10*10*64 => 10*10*64
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    # 10*10*64 => 6400
    model.add(Flatten())
    # 6400 => 512
    model.add(Dense(512))
    model.add(Activation('relu'))
    # 512 => 2
    model.add(Dense(2))
    
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    model.summary()
    return model

In [11]:
buildmodel()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_5 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 64)        32832     
_________________________________________________________________
activation_6 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
activation_7 (Activation)    (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
__________

C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(80, 80, 4..., strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  import sys
C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  # Remove the CWD from sys.path while we load stuff.


In [12]:
def playGame(args):
    model = buildmodel()
    trainNetwork(model,args)

In [ ]:
playGame({'mode':'Train'})

C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(80, 80, 4..., strides=(4, 4), padding="same")`
  after removing the cwd from sys.path.
C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), strides=(2, 2), padding="same")`
  import sys
C:\Users\davidchiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\davidchiu\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_13 (Activation)   (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        32832     
_________________________________________________________________
activation_14 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
activation_15 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6400)              0         
__________

TIMESTEP 78 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 79 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 80 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 81 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 82 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 83 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 84 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 85 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 86 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 87 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 88 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 89 / STATE observe / EPSILON 0.1 /

TIMESTEP 172 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 173 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 174 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 175 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 176 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 177 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 178 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 179 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 180 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 181 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 182 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 183 / STATE observe / E

TIMESTEP 264 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 265 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 266 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 267 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 268 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 269 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 270 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 271 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 272 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 273 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 274 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 275 / STATE observe / E

...

TIMESTEP 3189 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3190 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3191 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 3192 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3193 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3194 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3195 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3196 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3197 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3198 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3199 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD -1 / Q_MAX  

TIMESTEP 3255 / STATE explore / EPSILON 0.09999820180000035 / ACTION 1 / REWARD 0.1 / Q_MAX  18.085758 / Loss  1.2651742696762085
TIMESTEP 3256 / STATE explore / EPSILON 0.09999816850000036 / ACTION 1 / REWARD 0.1 / Q_MAX  18.170559 / Loss  1.4496244192123413
TIMESTEP 3257 / STATE explore / EPSILON 0.09999813520000037 / ACTION 1 / REWARD 0.1 / Q_MAX  18.857174 / Loss  1.0854852199554443
TIMESTEP 3258 / STATE explore / EPSILON 0.09999810190000037 / ACTION 1 / REWARD 0.1 / Q_MAX  19.634504 / Loss  6.573596954345703
TIMESTEP 3259 / STATE explore / EPSILON 0.09999806860000038 / ACTION 1 / REWARD 0.1 / Q_MAX  21.216034 / Loss  1.9995386600494385
TIMESTEP 3260 / STATE explore / EPSILON 0.09999803530000038 / ACTION 1 / REWARD 0.1 / Q_MAX  22.504627 / Loss  2.5077075958251953
TIMESTEP 3261 / STATE explore / EPSILON 0.09999800200000039 / ACTION 1 / REWARD 0.1 / Q_MAX  24.942663 / Loss  0.49181243777275085
TIMESTEP 3262 / STATE explore / EPSILON 0.0999979687000004 / ACTION 1 / REWARD 0.1 / Q_MAX

TIMESTEP 3318 / STATE explore / EPSILON 0.09999610390000076 / ACTION 1 / REWARD 0.1 / Q_MAX  56.25139 / Loss  8.605195045471191
TIMESTEP 3319 / STATE explore / EPSILON 0.09999607060000076 / ACTION 1 / REWARD 0.1 / Q_MAX  55.601402 / Loss  2.200312376022339
TIMESTEP 3320 / STATE explore / EPSILON 0.09999603730000077 / ACTION 1 / REWARD 0.1 / Q_MAX  54.799232 / Loss  1.343825340270996
TIMESTEP 3321 / STATE explore / EPSILON 0.09999600400000078 / ACTION 1 / REWARD 0.1 / Q_MAX  53.506596 / Loss  0.37368327379226685
TIMESTEP 3322 / STATE explore / EPSILON 0.09999597070000078 / ACTION 0 / REWARD 0.1 / Q_MAX  53.62468 / Loss  0.8203784227371216
----------Random Action----------
TIMESTEP 3323 / STATE explore / EPSILON 0.09999593740000079 / ACTION 0 / REWARD 0.1 / Q_MAX  55.3251 / Loss  2.449265241622925
TIMESTEP 3324 / STATE explore / EPSILON 0.0999959041000008 / ACTION 0 / REWARD 0.1 / Q_MAX  56.86899 / Loss  2.6136996746063232
TIMESTEP 3325 / STATE explore / EPSILON 0.0999958708000008 / ACTI

TIMESTEP 3381 / STATE explore / EPSILON 0.09999400600000116 / ACTION 1 / REWARD 0.1 / Q_MAX  182.06088 / Loss  26.203815460205078
TIMESTEP 3382 / STATE explore / EPSILON 0.09999397270000117 / ACTION 1 / REWARD 0.1 / Q_MAX  189.57764 / Loss  2.3682150840759277
TIMESTEP 3383 / STATE explore / EPSILON 0.09999393940000117 / ACTION 1 / REWARD 0.1 / Q_MAX  194.4551 / Loss  3.746716022491455
TIMESTEP 3384 / STATE explore / EPSILON 0.09999390610000118 / ACTION 1 / REWARD 0.1 / Q_MAX  202.35922 / Loss  12.022170066833496
TIMESTEP 3385 / STATE explore / EPSILON 0.09999387280000119 / ACTION 0 / REWARD 0.1 / Q_MAX  204.82507 / Loss  5.896351337432861
TIMESTEP 3386 / STATE explore / EPSILON 0.0999938395000012 / ACTION 0 / REWARD 0.1 / Q_MAX  210.12917 / Loss  92.43301391601562
TIMESTEP 3387 / STATE explore / EPSILON 0.0999938062000012 / ACTION 0 / REWARD 0.1 / Q_MAX  214.97562 / Loss  8.560744285583496
TIMESTEP 3388 / STATE explore / EPSILON 0.09999377290000121 / ACTION 0 / REWARD 0.1 / Q_MAX  226.

TIMESTEP 3443 / STATE explore / EPSILON 0.09999194140000156 / ACTION 0 / REWARD 0.1 / Q_MAX  236.09064 / Loss  7.358635902404785
TIMESTEP 3444 / STATE explore / EPSILON 0.09999190810000157 / ACTION 0 / REWARD 0.1 / Q_MAX  230.32375 / Loss  12.367966651916504
TIMESTEP 3445 / STATE explore / EPSILON 0.09999187480000157 / ACTION 0 / REWARD 0.1 / Q_MAX  231.21245 / Loss  6.698103904724121
TIMESTEP 3446 / STATE explore / EPSILON 0.09999184150000158 / ACTION 0 / REWARD 0.1 / Q_MAX  228.82191 / Loss  19.116863250732422
TIMESTEP 3447 / STATE explore / EPSILON 0.09999180820000159 / ACTION 0 / REWARD 0.1 / Q_MAX  223.07774 / Loss  28.68067169189453
TIMESTEP 3448 / STATE explore / EPSILON 0.09999177490000159 / ACTION 1 / REWARD 0.1 / Q_MAX  222.62315 / Loss  14.715510368347168
TIMESTEP 3449 / STATE explore / EPSILON 0.0999917416000016 / ACTION 1 / REWARD 0.1 / Q_MAX  273.66736 / Loss  16.425439834594727
TIMESTEP 3450 / STATE explore / EPSILON 0.0999917083000016 / ACTION 1 / REWARD 0.1 / Q_MAX  21